## Deep Neural Network for MNIST Classification
We'll aplly all the knowledge from the lectures in this section to write a deep neural network. The problem we've chosen is referred to as the "Hello World" of deep learning because for the most studants it is the first deep learning algorithm they see.<br /><br />
The dataset is called MNIST and refers to handwritten digit reconition. You can find more about it on Yann LeCun's website (Cirector of AI Research, Facebook). He's one of the pioneers of what we've been talking about and of more complex approaches that are widely used today, such as covolutional neural networks (CNNs).<br /><br />
The dataset provides 70.000 images (28x28 pixels) of handwritten digits (1 digit per image).<br /><br />
The goal is to write an algorithm that detects which digit written. Since there are only 10 digits (0, 1, 2, 3, 4, 5, 6, 7, 8, 9), this is a classification problem with 10 classes.<br /><br />
Our goal would be to build a neural network with 2 hidden layers.
### Import the relevant packages

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

### Data

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True, data_dir='tds_Data')

In [3]:
mnist_info

tfds.core.DatasetInfo(
    name='mnist',
    version=3.0.0,
    description='The MNIST database of handwritten digits.',
    homepage='http://yann.lecun.com/exdb/mnist/',
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=70000,
    splits={
        'test': 10000,
        'train': 60000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann. lecun. com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
    redistribution_info=,
)

In [4]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

n_valid_sample = 0.1 * mnist_info.splits['train'].num_examples
n_valid_sample = tf.cast(n_valid_sample, tf.int64)

n_test_sample = mnist_info.splits['test'].num_examples
n_test_sample = tf.cast(n_test_sample, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_valid_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [5]:
BUFFER_SIZE = 10000
shuffled_train_and_valid_data = scaled_train_and_valid_data.shuffle(BUFFER_SIZE)

valid_data = shuffled_train_and_valid_data.take(n_valid_sample)
train_data = shuffled_train_and_valid_data.skip(n_valid_sample)

BATCH_SIZE = 150
train_data = train_data.batch(BATCH_SIZE)
valid_data = valid_data.batch(n_valid_sample)
test_data = test_data.batch(n_test_sample)

valid_inputs, valid_targets = next(iter(valid_data))

## Model

### Outline the model

In [6]:
input_size = 784
output_size = 10
hidden_layer_size = 2000

model = tf.keras.Sequential([
    
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='tanh'),
    tf.keras.layers.Dense(hidden_layer_size, activation='sigmoid'),
    tf.keras.layers.Dense(hidden_layer_size, activation='hard_sigmoid'),
    tf.keras.layers.Dense(output_size,activation='softmax')
    
])

### Choose the optimizer and the loss function

In [7]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=0.0015
    ), 
    loss='sparse_categorical_crossentropy', 
    metrics =['accuracy']
)

### Training

In [8]:
N_EPOCHS = 10

model.fit(
    train_data, 
    epochs = N_EPOCHS, 
    validation_data=(
      valid_inputs, 
      valid_targets
    ),
    validation_steps=BATCH_SIZE, 
    verbose=2
)

Epoch 1/10
360/360 - 64s - loss: 1.3674 - accuracy: 0.5268 - val_loss: 0.2846 - val_accuracy: 0.9305
Epoch 2/10
360/360 - 57s - loss: 0.2013 - accuracy: 0.9474 - val_loss: 0.1588 - val_accuracy: 0.9580
Epoch 3/10
360/360 - 55s - loss: 0.1145 - accuracy: 0.9686 - val_loss: 0.0901 - val_accuracy: 0.9725
Epoch 4/10
360/360 - 55s - loss: 0.0819 - accuracy: 0.9773 - val_loss: 0.0665 - val_accuracy: 0.9810
Epoch 5/10
360/360 - 52s - loss: 0.0615 - accuracy: 0.9823 - val_loss: 0.0633 - val_accuracy: 0.9823
Epoch 6/10
360/360 - 52s - loss: 0.0464 - accuracy: 0.9865 - val_loss: 0.0443 - val_accuracy: 0.9878
Epoch 7/10
360/360 - 51s - loss: 0.0374 - accuracy: 0.9890 - val_loss: 0.0414 - val_accuracy: 0.9885
Epoch 8/10
360/360 - 51s - loss: 0.0353 - accuracy: 0.9896 - val_loss: 0.0305 - val_accuracy: 0.9908
Epoch 9/10
360/360 - 51s - loss: 0.0274 - accuracy: 0.9917 - val_loss: 0.0344 - val_accuracy: 0.9900
Epoch 10/10
360/360 - 51s - loss: 0.0250 - accuracy: 0.9923 - val_loss: 0.0299 - val_accura

In [9]:
test_loss, test_accuracy = model.evaluate(test_data)

      1/Unknown - 3s 3s/step - loss: 0.0827 - accuracy: 0.9810

In [10]:
# We can apply some nice formatting if we want to
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.08. Test accuracy: 98.10%
